In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedKFold, cross_val_score

In [2]:
train_filepath = '/kaggle/input/titanic/train.csv'
test_filepath = '/kaggle/input/titanic/test.csv'

In [3]:
drop_feature_names = ['Name', 'Cabin', 'Ticket']

In [4]:
map_features = ['Sex', 'Embarked']
map_dicts = [{'male':0, 'female':1}, {'Q':0, 'C':1, 'S':2}]

In [5]:
models = []

models.append(('SVM', SVC()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('LR', LogisticRegression()))

In [6]:
def drop_features(df, d_features):
    df_dropped = df
    
    for feature in d_features:
        df_dropped = df_dropped.drop(str(feature), axis=1)
        
    return df_dropped

In [7]:
def map_cat(df, features, dictionaries):
    '''
    Map categorical data to be numeric
    
    Input:
        - Dataframe for the mapping to take place in
        - List of features that contain categorical data
        - List of dictionaries to govern how to map the feature
    
    Output: A dataframe with the given features mapped according to the dictionaries
        
    '''
    df_mapped = df
    
    for i in range(len(features)):
        feature = features[i]
        dictionary = dictionaries[i]
        
        df_mapped[feature] = df[feature].map(dictionary)
        
    return df_mapped

In [8]:
def normalize(feature):
    '''
    This function normalizes a Series
    
    Input: A feature of type Series
    
    Output: The normalized feature of type Series
    '''
    return (feature - feature.mean())/feature.std()

In [9]:
def normalize_features(df):
    '''
    This function normalizes all features in a dataframe
    
    Input: A pandas dataframe
    
    Output: The normalized dataframe
    '''
    for column in df.columns:
        df[column] = normalize(df[column])
    return df

In [10]:
def perform_cross_validation(X_train, y_train):
    '''
    This function goes through the steps to perform Stratified K-fold cross validation using the list of models described above.
    
    Input: 
        - A dataframe containing the features use to build the model
        - A Series of the true values associated with the feature list
    
    Output: Printed result for the mean and standard deviation of each model
    '''
    results = dict()

    for name, model in models:
        kfold = StratifiedKFold(n_splits=10)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
        results[name] = (cv_results.mean(), cv_results.std())

    print('Model\t\tCV Mean\t\tCV std')
    print(results)

In [11]:
df_train = pd.read_csv(train_filepath, index_col='PassengerId')
df_test = pd.read_csv(test_filepath, index_col='PassengerId')

In [12]:
drop_feature_names_w_age = drop_feature_names + ['Age']

In [13]:
df_train = drop_features(df_train, drop_feature_names_w_age)
df_test = drop_features(df_test, drop_feature_names_w_age)

In [14]:
df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].mean())

In [15]:
df_train['Embarked'] = df_train['Embarked'].fillna('S')

In [16]:
df_train = map_cat(df_train, map_features, map_dicts)
df_test = map_cat(df_test, map_features, map_dicts)

In [17]:
X = df_train.drop('Survived', axis=1)
y = df_train['Survived']

In [18]:
X = normalize_features(X)
df_test = normalize_features(df_test)

In [19]:
perform_cross_validation(X, y)

Model		CV Mean		CV std
{'SVM': (0.8058052434456927, 0.02817776516195646), 'KNN': (0.8092384519350813, 0.04749914265913915), 'LR': (0.7934706616729088, 0.024861946815253916)}


In [20]:
clf = KNeighborsClassifier().fit(X,y)

predictions = clf.predict(df_test)

submission1 = pd.DataFrame(data={'Survived':predictions}, index=df_test.index)

submission1.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [21]:
submission1.to_csv('Titanic_Submission-No_Age-2023_6_14.csv')